In [ ]:
from pydantic import BaseModel


class Cube(BaseModel):
    x: int
    y: int
    z: int

    def __repr__(self) -> str:
        return f"({self.x},{self.y},{self.z})"


class Brick(BaseModel):
    cubes: list[Cube]
    id: int

    def lowest_z_value(self) -> int:
        return min(c.z for c in self.cubes)

    def drop_to_z_value(self, z: int) -> None:
        fall_height = self.lowest_z_value() - z
        for cube in self.cubes:
            cube.z = cube.z - fall_height

    @classmethod
    def from_string(cls, string: str, id: int) -> "Brick":
        start, end = string.split("~")
        start = tuple(map(int, start.split(",")))
        end = tuple(map(int, end.split(",")))
        cubes = []
        for x in range(start[0], end[0] + 1):
            for y in range(start[1], end[1] + 1):
                for z in range(start[2], end[2] + 1):
                    cubes.append(Cube(x=x, y=y, z=z))
        return cls(cubes=cubes, id=id)

In [ ]:
# Read initial positions from file

bricks: list[Brick] = []
with open("day22_input.txt") as file:
    for i, line in enumerate(file):
        bricks.append(Brick.from_string(line.strip(), i))

In [ ]:
# Let all the bricks settle in their lowest possible position

grid = dict()
for brick in sorted(bricks, key=lambda b: b.lowest_z_value()):
    # Can we lower the brick?
    current_level = brick.lowest_z_value()
    drop_to_level = [1 for _ in brick.cubes]
    for i, cube in enumerate(brick.cubes):
        for z in range(current_level, 0, -1):
            if (cube.x, cube.y, z) in grid:
                drop_to_level[i] = z + 1
                break

    # Drop this brick to its lowest possible level
    brick.drop_to_z_value(max(drop_to_level))

    # Put the brick cubes into common grid
    for cube in brick.cubes:
        grid[(cube.x, cube.y, cube.z)] = brick.id

In [ ]:
# Figure out which bricks are supported by which other bricks

from collections import defaultdict

supported_by = defaultdict(set)
is_supporting = defaultdict(set)
for brick in bricks:
    for cube in brick.cubes:
        above = (cube.x, cube.y, cube.z + 1)
        if (above in grid) and (grid[above] != brick.id):
            is_supporting[brick.id].add(grid[above])
            supported_by[grid[above]].add(brick.id)

# Part 1


In [ ]:
safe_to_remove = 0

# For each brick
for brick in bricks:
    # For each other brick being supported by this brick
    for supported in is_supporting[brick.id]:
        # Are we the only brick supporting that brick?
        if supported_by[supported] == set([brick.id]):
            break
    else:
        safe_to_remove += 1

print("Answer:", safe_to_remove)

# Part 2


In [ ]:
from collections import deque

would_fall = 0

# For each brick
for brick in bricks:
    fallen = set()
    queue = deque([brick.id])
    while queue:
        removing = queue.popleft()
        # What other bricks are supported by this brick?
        for supported in is_supporting[removing]:
            # Have all supporting bricks fallen?
            if len(supported_by[supported] - {brick.id} - fallen) == 0:
                if supported not in fallen:
                    fallen.add(supported)
                    queue.append(supported)

    # We're done with the chain reaction for `brick`. How many bricks fell?
    would_fall += len(fallen)

print("Answer:", would_fall)